In [1]:
import numpy as np
import h5py

import matplotlib.pyplot as plt
import pywt

import pycwt

### 从hdf5文件中读取窗口化的一维时序数据


In [2]:
# 打开一维数据文件

noise_dataset = h5py.File(r'F:\PythonProjects\CCLSN\2-CWT\inputdata\TSH_noise.h5', "r")
event_dataset = h5py.File(r'F:\PythonProjects\CCLSN\2-CWT\inputdata\TSH_event.h5', "r")

In [3]:
# 读取文件数据

noise_data = np.array(noise_dataset["noise"][:])
event_data = np.array(event_dataset["event"][:])

In [5]:
# 验证数组结构，确保程序正常运行

event_data.shape

(1556, 2501, 3)

In [6]:
# 验证数组结构，确保程序正常运行

noise_data.shape

(1440, 2501, 3)

## 这一部分为数据直接做小波变换和下采样后储存

### 每一个样本的三分量一维时序数据分别做小波变换再堆叠成三分量的时频二维矩阵

输入的时序数据数组的结构为**(m,n,3)**，m为样本个数，n为时序数据长度，3为3分量维度
时序数据经过小波变换后转化为时频矩阵，此时，整个数据数组的结构变为
**(m,f,t,3)**，m为样本个数，f为频域，t为时域，3为3分量维度。


主要功能函数由自编扩展包 **pycwt** 实现，主要功能函数有:
+ **cwt4series(data,wavename = 'cmor3-3',totalscal = 46,sampling_rate = 100)** :一维时序数据进行小波时频变换
+ **subs_matr(cwtmatr,n = 1,m = 10)** :对变换后的二维时频矩阵进行下采样，**下采样因子（即缩小倍数）频域为n，默认为 n = 1 ，时域为m，默认为 m = 10。**

In [7]:
# 数据采样率，本程序取原始采样率100Hz
# sampling_rate = 100

In [8]:
# 每个窗口的时间序列，本程序的窗长为25.01s
# time = 25.01
# t = np.arange(0, time, 1.0 / sampling_rate)

In [11]:
# 小波变换所采用的的小波
# 此为小波变换的关键参数之一，本程序采用cmor小波，参数分别取3,3

# wavename = 'cmor3-3' 为cwt4series()默认小波

In [ ]:
# 小波变换后频率域的取值，小波变化后频率域的分布为0-（totalscal-1）
# 小波变换频域的取值范围主要由地震事件的性质和仪器的相应频谱决定
# 程序应用时，该参数需要根据实际情况做调整
# 本程序默认取46，小波变换后的频域为0-45Hz

# totalscal = 46 为cwt4series()默认频域

In [ ]:
# 对事件窗口进行小波变换

DATA_e = []

for i in np.arange(event_data.shape[0]):
    data = []
    for j in [0,1,2]:
        event = event_data[i,:,j]
        cwt = pycwt.cwt4series(event,sampling_rate = 100)
        cwtmatr = pycwt.subs_matr(cwt[0])
        data.append(cwtmatr)
    data = np.array(data)
    data = np.stack(data, axis=2)
    DATA_e.append(data)
    
DATA_e = np.array(DATA_e)

In [ ]:
# 确保输出的数组结构正确

DATA_e.shape

In [ ]:
# 对噪声窗口进行小波变换

DATA_n = []

for i in np.arange(noise_data.shape[0]):
    data = []
    for j in [0,1,2]:
        noise = noise_data[i,:,j]
        cwt = pycwt.cwt4series(noise,sampling_rate = 100)
        cwtmatr = pycwt.subs_matr(cwt[0])
        data.append(cwtmatr)
    data = np.array(data)
    data = np.stack(data, axis=2)
    DATA_n.append(data)
    
DATA_n = np.array(DATA_n)

In [ ]:
# 确保输出的数组结构正确

DATA_n.shape

### 将小波时频变换后的数组保存到hdf5文件

便于机器学习模型读取

In [ ]:
with h5py.File(r'QKT_week1_cwt.h5','a') as f:
    f.create_dataset('event',data = DATA_e)
    f.create_dataset('noise',data = DATA_n)

### 关闭打开的数据文件

In [ ]:
noise_dataset.close()
event_dataset.close()

## 这一部分为选择特定数据作小波变换后绘图展示

#### 事件窗口绘图

In [ ]:
# 选择事件和方向分量
event = event_data[3,:,0]

[cwt, frequencies] = pycwt.cwt4series(event,sampling_rate = 100)
cwtmatr = pycwt.subs_matr(cwt)

In [ ]:
t = np.arange(0, 40, 1.0 / 100)

t1 = np.arange(0, 40, 1.0 / 10)

In [ ]:
plt.figure(figsize=(8, 16))
plt.subplot(211)
plt.plot(t, event)

plt.subplot(212)
plt.contourf(t1, frequencies, abs(cwtmatr))
plt.show()

#### 噪声窗口绘图

In [ ]:
# 选择事件和方向分量
noise = noise_data[3,:,0]

[cwt, frequencies] = pycwt.cwt4series(noise,sampling_rate = 100)
cwtmatr = pycwt.subs_matr(cwt)

In [ ]:
plt.figure(figsize=(8, 16))
plt.subplot(211)
plt.plot(t, noise)

plt.subplot(212)
plt.contourf(t1, frequencies, abs(cwtmatr))
plt.show()

### 关闭打开的数据文件

In [ ]:
noise_dataset.close()
event_dataset.close()